# 2-6 損失関数の実装

## jaccard 係数を用いた match 関数の動作
はじめに，すべてのデフォルトボックスから正解データと物体クラスが一致しており座標情報の近いものを抽出する関数 match を定義する．
正解のバウンディングボックスと近いデフォルトボックスの抽出には jaccard 係数を用いる．
jaccard 係数は次の図のように計算できる．

<img src="../image/p105_1.png">

バウンディングボックスとデフォルトボックスの間の jaccard 係数は，2つのボックスの総面積に対するかぶっている部分の面積で計算できる．
2つのボックスが完全にかぶっていれば jaccard 係数は1に，完全に外れている場合には0となる．  
この jaccard 係数を用いて訓練データの正解バウンディングボックスとの係数の値が閾値（ここでは jaccard_thresh = 0.5）以上のデフォルトボックスを Positive Default Box とする．

<img src="../image/p105_2.png">

実装においては次のように処理を行う．
まず，Positive なデフォルトボックスがない場合，何もない部分を背景として認識させるため，背景ラベルである0を与える．  
jaccard 係数が閾値以上のデフォルトボックスだある場合，それを Positive として jaccard 係数が最大になる正解バウンディングボックスの物体クラスを教師データの正解クラスとし，Positive なデフォルトボックスから正解のバウンディングボックスに変形させるオフセット値を loc の教師データとする．
SSD ではデフォルトボックスの座標情報と検出した物体クラスを別々に扱っている点に注意する必要がある．  
match 関数の実装はかなり複雑なため，実装済みの"./utils/match.py"を流用する．
オフセット情報とラベル情報の正解教師データとして，それぞれ loc_t と conf_t_label を返すようになっている．
背景クラスをインデックス0とするため，VOC2012 で用意されている物体クラスのインデックスを+1している．

## Hard Negative Mining
損失値を計算する前に Hard Negative Mining 処理を行う．
この処理は Negative に分類されたデフォルトボックスのうち，学習に使用するデフォルトボックスの数を絞る操作である．  
教師データの loc_t は Positive と判定されたデフォルトボックスにのみ用意されているが，conf_t_label は全てのデフォルトボックスに対して用意されている．
デフォルトボックスのうち Negative と判定されるもののほうが圧倒的に多く，全てのデフォルトボックスを学習に用いると，ほとんどのラベルが背景（0）となり物体クラスの学習回数が相対的に少なくなってしまう．
これを防ぐために，Negative なデフォルトボックスの数を Positive の一定数倍（ここでは3倍）に制限する．  
このとき選択する Negative なデフォルトボックスは，ラベル予測の損失値が大きいもの，すなわち背景クラスであるべきなのに正しく背景クラスと予測できていないものを優先的に選択する．

## SmoothL1Loss 関数と交差エントロピー誤差関数
オフセット情報 loc の予測はデフォルトボックスから正解のバウンディングボックスに変換するための補正値に関する回帰問題である．
通常，回帰問題の損失関数には二乗誤差が用いられるが，ここでは次式で定義される SmoothL1Loss 関数を用いる．

$$
    loss_i(loc_t - loc_p) = 
    \begin{cases}
        {0.5\times(loc_t - loc_p)^2 \qquad if |loc_t - loc_p| < 1}\\\
        {|loc_t - loc_p| - 0.5 \qquad otherwise}   
    \end{cases}
$$

二乗誤差を使うと教師データと予測結果の差が大きい場合に，異常に値が大きくなり学習が不安定になりやすい．
そのため，教師データと予測結果の差が大きいときには，その絶対値を用いることで損失が大きくなりすぎるのを防ぐ．  
また，オフセットの予測に関しては，背景クラスは教師データとなるバウンディングボックスが存在しないため，Positive と判定されたもののみを用いる．

物体のラベル予測に関しては，次式で定義される交差エントロピー誤差関数を用いる

$$
    loss_i(conf, label_t) = -log\left(\frac{exp(conf[label_t]}{\Sigma exp(conf[x])}\right)
$$

## SSD の損失関数クラス MultiBoxLoss の実装
これまで説明してきた事項を踏まえて MultiBoxLoss クラスを実装する．
高速処理のため直感的でないコードも含まれるため，概念的な理解を優先しコードの詳細はあとからでも良い．

In [2]:
import torch
from torch import nn
from utils.match import *
import torch.nn.functional as F

class MultiBoxLoss(nn.Module):
    ''' SSD の損失関数を計算するクラス '''
    
    def __init__(self, jaccard_threshold=0.5, neg_pos=3, device='cpu'):
        super(MultiBoxLoss, self).__init__()
        self.jaccard_threshold = jaccard_threshold  # match 関数の jaccard 係数の閾値
        self.negpos_ratio = neg_pos  # Hard Negative Mining の制限値
        self.device = device    # CPU or GPU で計算
        
        
    def forward(self, predictions, targets):
        """
        損失関数の計算
        
        Parameters
        ----------
        predictions : (loc, conf, dbox_list)
            SSD の訓練時の出力
            loc : torch.Size([num_batch , 8732, 4])
            conf : torch.Size([num_batch , 8732, 21])
            dbox_list : torch.Size([8732, 4])
        targets : [num_batch , num_objs, 5]
            5 は正解のアノテーション情報 [xmin, ymin, xmax, ymax, label_ind]
            
        Returns
        -------
        loss_l : tensor
            loc の損失値
        loss_c : tensor
            conf の損失値
        """
        
        # SSD モデルの出力はタプルなので分解
        loc_data, conf_data, dbox_list = predictions
        
        # 要素数のカウント
        num_batch = loc_data.size(0)    # ミニバッチサイズ
        num_dbox = loc_data.size(1)     # デフォルトボックスの数 = 8732
        num_classes = conf_data.size(2) # クラス数 = 21
        
        # 損失の計算に使用するものを格納する変数を作成
        # 各デフォルトボックスに一番近い正解のバウンディングボックスのラベルを格納させる
        conf_t_label = torch,LongTensor(num_batch, num_dbox).to(self.device)
        # 各デフォルトボックスに一番近い正解のバウンディングボックスの位置情報を格納させる
        loc_t = torch.Tensor(num_batch, num_dbox, 4).to(self.device)
        
        # loc_t と conf_t_label にデフォルトボックスと targets を match させた結果を上書き
        for idx in range(num_batch):
            # 現在のミニバッチの正解アノテーションのバウンディングボックスとラベルを取得
            truths = targets[idx][:, :-1].to(self.device)
            labels = targets[idx][:, -1].to(self.device)
            
            dbox = dbox_list.to(self.device)
            # 関数 match を実行し、loc_t と conf_t_label の内容を更新する
            # loc_t: 各デフォルトボックスに一番近い正解のバウンディングボックスの位置情報が上書きされる
            # conf_t_label:各デフォルトボックスに一番近いバウンディングボックスのラベルが上書きされる
            # ただし、一番近いバウンディングボックスとの jaccard overlap が 0.5 より小さい場合は正解バウンディングボックスのラベル conf_t_label は背景クラスの 0 とする
            variance = [0.1, 0.2]  # バウンディングボックスに変換するときの係数
            match(self.jaccard_threshold, truths, dbox, variance, labels, loc_t, conf_t_label, idx)
            
        #---------------------------------------------------------------------------------------------------
        # 位置の損失:loss_l を計算
        # Smooth L1 関数で損失を計算する。ただし、物体を発見したデフォルトボックスのオフセットのみを計算する
        #---------------------------------------------------------------------------------------------------
        
        # 物体を検出したバウンディングボックスを取り出すマスクを作成
        pos_mask = conf_t_label > 0
        # pos_mask を loc_data のサイズに変形
        pos_index = pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data)
        # Positive なデフォルトボックスと教師データ loc_t を取得
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        # Positive なデフォルトボックスの損失（誤差）を計算
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')
        
        #------------------------------------------------------------------------------------------------------------------
        # クラス予測の損失 loss_c の計算
        # 交差エントロピー誤差関数で損失を計算する。ただし、背景クラスが正解であるデフォルトボックスが圧倒的に多いので、
        # Hard Negative Mining を実施し、物体発見デフォルトボックスと背景クラスデフォルトボックスの比が 1:3 になるようにする
        # そこで背景クラスデフォルトボックスと予想したもののうち、損失が小さいものは、クラス予測の損失から除く
        # ------------------------------------------------------------------------------------------------------------------
        batch_conf = conf_data.view(-1, num_classes)
        
        # クラス予測の損失を関数を計算（reduction='none' にして、和をとらず次元をつぶさない）
        loss_c = F.cross_entropy(batch_conf, conf_t_label.view(-1), reduction="none")
        
        # Hard Negative Mining で抽出するデフォルトボックスのマスクを作成
        # Positive なデフォルトボックスの損失を0にする
        num_pos = pos_mask.long().sum(1, keepdim=True) # ミニバッチごとの物体クラス予測の数
        loss_c = loss.view(num_batch, -1)              # torch.Size([num_batch, 8732])
        loss_c[pos_mask] = 0                           # 物体のあるデフォルトボックスは損失を0にする
        
        # Hard Negative Mining を実行
        # 各デフォルトボックスの損失の大きさ loss_c の順位 idx_rank を求める
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        
        # Hard Negative Mining で残すデフォルトボックスの数を決める
        num_neg = torch.clamp(num_pos * self.negpos_ratio, max=num_dbox)
        
        # num_neg よりも損失の大きいデフォルトボックスを抽出するマスクを作る
        # torch.Size([num_batch, 8732])
        neg_mask < idx_rank < (num_neg).expand_as(idx_rank)
        
        # マスクの形を conf_data に揃える（torch.Size([num_batch, 8732]) => torch.Size([num_batch, 8732, 21])）
        pos_idx_mask = pos_mask.unsqueeze(2).expand_as(conf_data)  # Positive なデフォルトボックスの conf を取り出すマスク
        neg_idx_mask = neg_mask.unsqueeze(2).expand_as(conf_data)  # Negative なデフォルトボックスの conf を取り出すマスク
        
        # conf_data から pos と neg だけを取り出して conf_hnm とする（torch.Size([num_pos + num_neg, 21）
        conf_hnm = conf_data[(pos_idx_mask + neg_idx_mask).gt(0)].view(-1, num_classes)
        
        # conf_t_label から pos と neg だけを取り出して conf_t_label_hnm とする
        conf_t_label_hnm = conf_t_label[(pos_mask + neg_mask).gt(0)]
        
        # confidence の損失関数を計算
        loss_c = F.cross_entropy(conf_hnm, conf_t_label_hnm, reduction="sum")
        
        # 物体を発見したバウンディングボックスの数（全ミニバッチの合計）で損失を割り算
        N = num_pos.sum()
        loss_l /= N
        loss_c /= N
        
        return loss_l, loss_c

In [3]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', '2-6_Loss_Function.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', '2-6_Loss_Function.ipynb'], returncode=0)